In [2]:
import numpy as np
import pandas as pd

In [3]:
import gc

In [4]:
aisles_df = pd.read_csv('../data/original/aisles.csv')
departments_df = pd.read_csv('../data/original/departments.csv')

order_products_prior_df = pd.read_csv('../data/original/order_products__prior.csv')
order_products_train_df = pd.read_csv('../data/original/order_products__train.csv')

orders_df = pd.read_csv('../data/original/orders.csv')
products_df = pd.read_csv('../data/original/products.csv')

In [3]:
order_count_df = orders_df[orders_df['eval_set'] == 'prior'].groupby('user_id').count()

In [4]:
user_feature_df = pd.DataFrame(index=order_count_df.index)

In [5]:
user_feature_df['order_count'] = order_count_df['order_id']

In [6]:
base_feature_df = pd.merge(order_products_prior_df, orders_df, how='left', on="order_id")

In [7]:
base_feature_df = pd.merge(base_feature_df, products_df, how='left', on='product_id')

In [8]:
base_feature_df = pd.merge(base_feature_df, products_df, how='left', on='product_id')

In [9]:
base_feature_df

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name_x,aisle_id_x,department_id_x,product_name_y,aisle_id_y,department_id_y
0,2,33120,1,1,202279,prior,3,5,9,8.0,Organic Egg Whites,86,16,Organic Egg Whites,86,16
1,2,28985,2,1,202279,prior,3,5,9,8.0,Michigan Organic Kale,83,4,Michigan Organic Kale,83,4
2,2,9327,3,0,202279,prior,3,5,9,8.0,Garlic Powder,104,13,Garlic Powder,104,13
3,2,45918,4,1,202279,prior,3,5,9,8.0,Coconut Butter,19,13,Coconut Butter,19,13
4,2,30035,5,0,202279,prior,3,5,9,8.0,Natural Sweetener,17,13,Natural Sweetener,17,13
5,2,17794,6,1,202279,prior,3,5,9,8.0,Carrots,83,4,Carrots,83,4
6,2,40141,7,1,202279,prior,3,5,9,8.0,Original Unflavored Gelatine Mix,105,13,Original Unflavored Gelatine Mix,105,13
7,2,1819,8,1,202279,prior,3,5,9,8.0,All Natural No Stir Creamy Almond Butter,88,13,All Natural No Stir Creamy Almond Butter,88,13
8,2,43668,9,0,202279,prior,3,5,9,8.0,Classic Blend Cole Slaw,123,4,Classic Blend Cole Slaw,123,4
9,3,33754,1,1,205970,prior,16,5,17,12.0,Total 2% with Strawberry Lowfat Greek Strained...,120,16,Total 2% with Strawberry Lowfat Greek Strained...,120,16


In [10]:
product_feature_df = base_feature_df.groupby(['user_id', 'product_id'])['order_id'].count().reset_index()

In [11]:
product_feature_df = product_feature_df.rename(columns={'order_id':'product_count'})

In [12]:
product_feature_df['product_buy_rate'] = (base_feature_df.groupby(['user_id', 'product_id'])['order_id'].count() / user_feature_df['order_count']).values

In [13]:
product_feature_df = pd.merge(product_feature_df, products_df, how='left', on='product_id')

In [14]:
product_feature_df['add_cart_order_mean'] = (base_feature_df.groupby(['user_id', 'product_id'])['add_to_cart_order']).mean().values
product_feature_df['avg_add_cart_order_std'] = (base_feature_df.groupby(['user_id', 'product_id'])['add_to_cart_order']).std().values

In [15]:
product_feature_df['avg_add_cart_order_std'].fillna(0,inplace=True)

In [16]:
product_count_per_order_df = pd.DataFrame(base_feature_df.groupby(['user_id', 'order_id']).count().groupby('user_id')['product_id'].mean()).reset_index()

In [17]:
product_count_per_order_df.rename(columns={'product_id':'product_count_per_order_mean'}, inplace=True)

In [18]:
product_feature_df = pd.merge(product_feature_df, product_count_per_order_df, how='left', on='user_id')

In [50]:
del product_count_per_order_df

In [20]:
def calculateBoughtNumberOfTimePeriodMean(order_number_series):
    if order_number_series.count() <= 1:
        return 100
    else:
        rate = (order_number_series.max() - order_number_series.min()) / (order_number_series.count() - 1)
        return rate

In [21]:
def calculateBoughtNumberOfTimePeriodStd(order_number_series):
    if order_number_series.count() <= 1:
        return 0
    else:
        sort_series = order_number_series.sort_values().reset_index(drop=True)
        intervals = []
        for i in range(1, sort_series.count()):
            intervals.append(sort_series[i] - sort_series[i - 1])
        return np.std(intervals)

In [22]:
bought_number_of_time_period_mean_df = base_feature_df.groupby(['user_id', 'product_id'])['order_number'].agg(calculateBoughtNumberOfTimePeriodMean)
bought_number_of_time_period_std_df = base_feature_df.groupby(['user_id', 'product_id'])['order_number'].agg(calculateBoughtNumberOfTimePeriodStd)

In [27]:
bought_number_of_time_period_mean_df = bought_number_of_time_period_mean_df.reset_index().rename(columns={'order_number':'order_mean_rate'})

In [34]:
bought_number_of_time_period_std_df = bought_number_of_time_period_std_df.reset_index().rename(columns={'order_number':'order_std_rate'})

In [37]:
product_feature_df = pd.merge(product_feature_df, bought_number_of_time_period_mean_df, how='left', on=['user_id', 'product_id'])

In [39]:
product_feature_df = pd.merge(product_feature_df, bought_number_of_time_period_std_df, how='left', on=['user_id', 'product_id'])

In [23]:
bought_length_of_time_period_mean_df = base_feature_df.groupby(['user_id', 'product_id'])['days_since_prior_order'].agg(np.mean)
bought_length_of_time_period_std_df = base_feature_df.groupby(['user_id', 'product_id'])['days_since_prior_order'].agg(np.std)

In [43]:
bought_length_of_time_period_mean_df = bought_length_of_time_period_mean_df.fillna(1000)
bought_length_of_time_period_mean_df = bought_length_of_time_period_mean_df.reset_index().rename(columns={'days_since_prior_order':'mean_days_since_prior_order'})

bought_length_of_time_period_std_df = bought_length_of_time_period_std_df.fillna(1000)
bought_length_of_time_period_std_df = bought_length_of_time_period_std_df.reset_index().rename(columns={'days_since_prior_order':'std_days_since_prior_order'})

In [46]:
product_feature_df = pd.merge(product_feature_df, bought_length_of_time_period_mean_df, how='left', on=['user_id', 'product_id'])
product_feature_df = pd.merge(product_feature_df, bought_length_of_time_period_std_df, how='left', on=['user_id', 'product_id'])

In [52]:
del bought_number_of_time_period_mean_df
del bought_number_of_time_period_std_df
del bought_length_of_time_period_mean_df
del bought_length_of_time_period_std_df

NameError: name 'bought_length_of_time_period_mean_df' is not defined

In [58]:
gc.collect()

600

,user_id,product_id,order_dow,order_number
0,1,196,1,3
1,1,196,2,2
2,1,196,3,2
3,1,196,4,3
4,1,10258,1,3
5,1,10258,2,1
6,1,10258,3,2
7,1,10258,4,3
8,1,10326,4,1
9,1,12427,1,3


In [10]:
def calculateOrderDowRate(order_dow_df):
    print(order_dow_df)

In [1]:
base_feature_df.groupby(['user_id', 'product_id']).agg(calculateOrderDowRate)

NameError: name 'base_feature_df' is not defined

In [12]:
test = {'A':[1, 2, 3, 4, 5, 2,4 ,5], 'B':['a', 'g', 'f', 'w', 'g', 'd', 'e', 'a'], 'C':[1,2,3,4,5,6,7,8], 'D':['a','b', 'c', 'd', 'e', 'f', 'g', 'h']}

test_df = pd.DataFrame(test)

In [14]:
test_df

,A,B,C,D
0,1,a,1,a
1,2,g,2,b
2,3,f,3,c
3,4,w,4,d
4,5,g,5,e
5,2,d,6,f
6,4,e,7,g
7,5,a,8,h


In [19]:
test_df.groupby(['A', 'B'])[['C', 'D']].transform(calculateOrderDowRate)

0    1
Name: C, dtype: object
0    a
Name: D, dtype: object
   C  D
0  1  a
5    6
Name: C, dtype: object
5    f
Name: D, dtype: object
1    2
Name: C, dtype: object
1    b
Name: D, dtype: object
2    3
Name: C, dtype: object
2    c
Name: D, dtype: object
6    7
Name: C, dtype: object
6    g
Name: D, dtype: object
3    4
Name: C, dtype: object
3    d
Name: D, dtype: object
7    8
Name: C, dtype: object
7    h
Name: D, dtype: object
4    5
Name: C, dtype: object
4    e
Name: D, dtype: object


,C,D
0,None,None
1,None,None
2,None,None
3,None,None
4,None,None
5,None,None
6,None,None
7,None,None


In [63]:
temp_df

,user_id,product_id,order_dow,order_number
0,1,196,1,3
1,1,196,2,2
2,1,196,3,2
3,1,196,4,3
4,1,10258,1,3
5,1,10258,2,1
6,1,10258,3,2
7,1,10258,4,3
8,1,10326,4,1
9,1,12427,1,3


In [64]:
pd.get_dummies(temp_df.drop(['order_number'], axis=1), columns=['order_dow'])

,user_id,product_id,order_dow_0,order_dow_1,order_dow_2,order_dow_3,order_dow_4,order_dow_5,order_dow_6
0,1,196,0,1,0,0,0,0,0
1,1,196,0,0,1,0,0,0,0
2,1,196,0,0,0,1,0,0,0
3,1,196,0,0,0,0,1,0,0
4,1,10258,0,1,0,0,0,0,0
5,1,10258,0,0,1,0,0,0,0
6,1,10258,0,0,0,1,0,0,0
7,1,10258,0,0,0,0,1,0,0
8,1,10326,0,0,0,0,1,0,0
9,1,12427,0,1,0,0,0,0,0


In [54]:
order_dow_df = pd.DataFrame(columns=['user_id'])

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name_x,aisle_id_x,department_id_x,product_name_y,aisle_id_y,department_id_y
0,2,33120,1,1,202279,prior,3,5,9,8.0,Organic Egg Whites,86,16,Organic Egg Whites,86,16
1,2,28985,2,1,202279,prior,3,5,9,8.0,Michigan Organic Kale,83,4,Michigan Organic Kale,83,4
2,2,9327,3,0,202279,prior,3,5,9,8.0,Garlic Powder,104,13,Garlic Powder,104,13
3,2,45918,4,1,202279,prior,3,5,9,8.0,Coconut Butter,19,13,Coconut Butter,19,13
4,2,30035,5,0,202279,prior,3,5,9,8.0,Natural Sweetener,17,13,Natural Sweetener,17,13
5,2,17794,6,1,202279,prior,3,5,9,8.0,Carrots,83,4,Carrots,83,4
6,2,40141,7,1,202279,prior,3,5,9,8.0,Original Unflavored Gelatine Mix,105,13,Original Unflavored Gelatine Mix,105,13
7,2,1819,8,1,202279,prior,3,5,9,8.0,All Natural No Stir Creamy Almond Butter,88,13,All Natural No Stir Creamy Almond Butter,88,13
8,2,43668,9,0,202279,prior,3,5,9,8.0,Classic Blend Cole Slaw,123,4,Classic Blend Cole Slaw,123,4
9,3,33754,1,1,205970,prior,16,5,17,12.0,Total 2% with Strawberry Lowfat Greek Strained...,120,16,Total 2% with Strawberry Lowfat Greek Strained...,120,16
